In [2]:
# import stuff
import pandas as p
import numpy as n
import seaborn as s

ModuleNotFoundError: No module named 'seaborn'

## Data Set 2; Stock Exchange Data

This is the data set containing the stock market information for 14 unique exchanges over the span of 12-31-1965 to 06-03-2021.

There are 3 csv files, but for this project, we will only be using indexData.csv. This has all the null values and extra data that indexProcessed.csv does not. 

[csv link](https://www.kaggle.com/mattiuzc/stock-exchange-data)


### Useful Stats:
- Size of set ~> 112,457
- Columns ~> 8

In [3]:
stock = p.read_csv('data/indexData.csv', header = None, skipinitialspace = True,
                encoding = 'latin-1', low_memory = False, skiprows=1,
                names = ['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
)
p.set_option('display.max_rows', 100)

In [4]:
stock

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [5]:
# To determine null values, and how many there are
stock.isna().sum()

Index           0
Date            0
Open         2204
High         2204
Low          2204
Close        2204
Adj Close    2204
Volume       2204
dtype: int64

A) The types of data being sampled are the values of 14 major stock indices from around the world. The tickers are listed below:

- NYA 
- IXIC
- HSI
- 000001.SS
- GSPTSE
- 399001.SZ
- NSEI
- GDAXI
- KS11
- SSMI
- TWII
- J203.JO
- N225
- N100

B) The features given by the data are the date, the open, close, and adjusted close. The high and low daily values as well as the volume are also given. These are general stats for all stock parket indices. 

C) 
### Qualitative & Nomial:
- Index

### Qualitative & Ordinal:
- Date

### Quantitative & Ordinal:
- Open
- High
- Low
- Close
- Adj Close

### Quantitative & Nomial:
- Volume

All *Quantitative* (except for Volume) values are continuous, while all *Qualitative* (Including Volume) values are discrete.

D) There are 2204 total null values in every quantitative column. This is due to potential dates that stocks did not happen, rather than neglect. Dates such as holidays or a certain day the stock market was closed. The ranges are given below with **NYA** being the lowest with 1 and **N225** being the highest with 626. This may make pivoting difficult. 

E) These feautures are chosen because as stated earlier, they are typical generic stock attributes and are very good for data analysis.

F) Some other features may be how many hours of trading were in each day. Some days are cut short for certain reasons. Other than that, not sure what else directly relates to the indices themselves. Maybe leader of the country at that time?

In [6]:
# G) Doing a pivot on this data

# Figuring out which dates have null values
stockNull = stock.isnull()
stockNull = stockNull.any(axis = 1)
stockNulls = stock[stockNull]
stockNulls

,Index,Date,Open,High,Low,Close,Adj Close,Volume
289,NYA,1967-02-23,NaN,NaN,NaN,NaN,NaN,NaN
26639,HSI,1987-01-01,NaN,NaN,NaN,NaN,NaN,NaN
26659,HSI,1987-01-29,NaN,NaN,NaN,NaN,NaN,NaN
26660,HSI,1987-01-30,NaN,NaN,NaN,NaN,NaN,NaN
26706,HSI,1987-04-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
108316,N100,2005-03-28,NaN,NaN,NaN,NaN,NaN,NaN
108511,N100,2005-12-26,NaN,NaN,NaN,NaN,NaN,NaN
110051,N100,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN
110643,N100,2014-05-01,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#sort the number of nulls
SNcount = p.DataFrame()
SNcount['Index'] = stockNulls[['Index']]
SNcount['Total Nulls'] = 0
SNcount = SNcount.groupby('Index').count()
SNcount = SNcount.sort_values('Total Nulls', ascending = False)

"""This is important because now we will know how many values will have NaN when we do our pivots."""
SNcount

,Total Nulls
Index,
N225,626
HSI,258
GSPTSE,250
000001.SS,172
399001.SZ,168
GDAXI,168
SSMI,159
KS11,152
TWII,141


In [8]:
#taking all non-null values and running a pivot on them
stock_exist = stock.dropna()

#This confirms they are all unique
print(len(stock_exist))
print(len(stock_exist[['Index', 'Date']].drop_duplicates()))

dates = ['2002-12-13', '2002-12-14', '2002-12-15', '2002-12-16', '2002-12-17']

stock_pivot = p.DataFrame()
for i in range(0,len(dates)):
    stock_pivot = stock_exist[['Index', 'Date', 'Adj Close']][stock_exist['Date'] == dates[i]]

stock_pivot.pivot(index = 'Date', columns = 'Index', values = 'Adj Close')

110253
110253


Index,000001.SS,399001.SZ,GDAXI,GSPTSE,HSI,IXIC,KS11,N100,N225,NYA,SSMI,TWII
Date,,,,,,,,,,,,
2002-12-17,1416.095947,2878.537842,3139.969971,6621.5,9715.740234,1392.050049,704.48999,552.549988,8510.730469,5094.740234,4902.299805,4545.603027


In [61]:
# PIVOT OF DATA FOR THE MONTH OF JANUARY 2021
date = stock_exist[(stock_exist['Date'] >= '2021-01-01') & (stock_exist['Date'] <= '2021-01-31')]

date.pivot(index = 'Date', columns = 'Index', values = 'Adj Close')


Index,000001.SS,399001.SZ,GDAXI,GSPTSE,HSI,IXIC,J203.JO,KS11,N100,N225,NSEI,NYA,SSMI,TWII
Date,,,,,,,,,,,,,,
2021-01-04,3502.958008,14827.46973,13726.74023,17527.80078,27472.81055,12698.45020,60479.42188,2944.449951,1112.520020,27258.38086,14132.90039,14376.70020,NaN,14902.03027
2021-01-05,3528.677002,15147.57031,13651.21973,17682.50000,27649.85938,12818.95996,60921.37109,2990.570068,1109.560059,27158.63086,14199.50000,14536.53027,10694.08984,15000.03027
2021-01-06,3550.876953,15187.61035,13891.96973,17828.09961,27692.30078,12740.79004,61857.42188,2968.209961,1120.229980,27055.93945,14146.25000,14788.83984,10747.08008,14983.12988
2021-01-07,3576.205078,15356.40039,13968.24023,18027.59961,27548.51953,13067.48047,63042.87891,3031.679932,1122.130005,27490.13086,14137.34961,14928.75000,10778.66992,15214.00000
2021-01-08,3570.110107,15319.29004,14049.53027,18042.09961,27878.22070,13201.98047,63519.17969,3152.179932,1132.099976,28139.02930,14347.25000,14966.83008,10797.99023,15463.95020
2021-01-11,3531.498047,15115.37988,13936.66016,17934.50000,27908.22070,13036.42969,63759.92969,3148.449951,1124.290039,NaN,14484.75000,14937.95996,10870.41992,15557.29981
2021-01-12,3608.340088,15460.03027,13925.05957,17985.80078,28276.75000,13072.42969,63535.10156,3125.949951,1122.560059,28164.33984,14563.45020,15010.29004,10875.25000,15500.70020
2021-01-13,3598.652100,15365.42969,13939.70996,17934.69922,28235.59961,13128.95020,63474.53125,3148.290039,1123.609985,28456.58984,14564.84961,14983.61035,10846.87988,15769.98047
2021-01-14,3565.905029,15070.12988,13988.70020,17958.09961,28496.85938,13112.63965,63884.76953,3149.929932,1133.199951,28698.25977,14595.59961,15044.37988,10850.80957,15707.19043
